In [ ]:
import json
from pathlib import Path
from collections import defaultdict

def read_annotation_metadata(anno_file):
    """
    Đọc metadata từ annotation JSON:
    - movie_title
    - year
    - clip_idx
    - frame_idx
    - resolution
    """
    
    with open(anno_file, 'r') as f:
        data = json.load(f)
    
    metadata = []
    
    for clip in data:
        movie_title = clip.get('movie_title', '')
        year = clip.get('year', '')
        clip_idx = clip.get('clip_idx', '')
        
        for frame in clip.get('key_frames', []):
            if 'frame_idx' not in frame:
                continue
                
            resolution = frame.get('resolution', [])
            
            # Lấy width, height từ resolution
            if len(resolution) == 2:
                width, height = resolution
            else:
                width, height = None, None
            
            metadata.append({
                'movie_title': movie_title,
                'year': year,
                'clip_idx': clip_idx,
                'width': width,
                'height': height
            })
            break
    
    return metadata

# Sử dụng
anno_file = "/home/serverai/ltdoanh/LayoutGeneration/objectfree/annotations/cmd_animated_movies_movienet.json"
metadata = read_annotation_metadata(anno_file)

# In ra 10 samples đầu
print(f"📊 Tổng số frames: {len(metadata)}")
print("\n🎬 Sample metadata:")
print("-" * 80)

for i, meta in enumerate(metadata[:10]):
    print(f"{i+1}. {meta['movie_title']}/{meta['year']}/{meta['clip_idx']}")
    print(f"   Resolution: {meta['width']}x{meta['height']}")

# Thống kê resolution distribution
print("\n📏 Resolution Distribution:")
resolution_counts = defaultdict(int)

for meta in metadata:
    if meta['width'] and meta['height']:
        res_key = f"{meta['width']}x{meta['height']}"
        resolution_counts[res_key] += 1

for res, count in sorted(resolution_counts.items(), key=lambda x: -x[1]):
    print(f"   {res}: {count} frames")

# Thống kê theo movie
print("\n🎥 Frames per movie:")
movie_counts = defaultdict(int)
for meta in metadata:
    movie_counts[meta['movie_

📊 Tổng số frames: 40

🎬 Sample metadata:
--------------------------------------------------------------------------------
1. Ice Age/2015/8dXF7y1QUxU
   Resolution: 640x360
2. Ice Age/2015/tQJTJdTM0Wk
   Resolution: 640x360
3. Sing/2017/bCOc7VCSox4
   Resolution: 640x344
4. Sing/2017/WDAlAqy9WUM
   Resolution: 640x344
5. Sing/2017/cUT0WQ9cTrg
   Resolution: 640x344
6. Sing/2017/lzJV7k-LiC4
   Resolution: 640x344
7. Sing/2017/ETC85CgzTHM
   Resolution: 640x344
8. Despicable Me/2015/YsMdkGG2b0k
   Resolution: 640x344
9. Despicable Me/2015/ej6dxNrh3Dc
   Resolution: 640x344
10. Despicable Me/2015/8R1OS5jPh2s
   Resolution: 640x344

📏 Resolution Distribution:
   640x344: 23 frames
   640x264: 14 frames
   640x360: 2 frames
   632x264: 1 frames

🎥 Frames per movie:
   Despicable Me: 7 frames
   Spider Man: Into the Spider-Verse: 7 frames
   Coraline: 6 frames
   Kung Fu Panda: 6 frames
   Sing: 5 frames
   The Secret Life of Pets: 5 frames
   Ice Age: 2 frames
   The Polar Express: 2 frames

In [9]:
# ...existing code...
import json
from pathlib import Path

def resolve_clip_key(name: str) -> str:
    stem = Path(name).stem  # Despicable Me_2015_x8HjCP3LqHo_1040
    parts = stem.split("_")
    if len(parts) < 4:
        raise ValueError(f"Cannot parse clip key from {name}")
    return f"{parts[0]}/{parts[1]}/{parts[2]}"

def scale_bbox_xyxy(bbox, sx, sy):
    x1, y1, x2, y2 = bbox
    return [x1 * sx, y1 * sy, x2 * sx, y2 * sy]

def scale_bbox_xywh(bbox, sx, sy):
    cx, cy, w, h = bbox
    return [cx * sx, cy * sy, w * sx, h * sy]

def process_annotations(root: Path, mapping: dict, suffix="_scaled"):
    for split in ("train", "val", "test"):
        ann_path = root / split / "annotations.json"
        if not ann_path.exists():
            continue
        data = json.loads(ann_path.read_text())
        for item in data:
            key = resolve_clip_key(item["image"])
            scales = mapping.get(key)
            if not scales:
                continue
            sx = scales["scale_x"]
            sy = scales["scale_y"]
            for box in item.get("boxes", []):
                if "bbox_xyxy" in box:
                    box["bbox_xyxy"] = scale_bbox_xyxy(box["bbox_xyxy"], sx, sy)
                if "bbox_xywh" in box:
                    box["bbox_xywh"] = scale_bbox_xywh(box["bbox_xywh"], sx, sy)
        out_path = ann_path.with_stem(ann_path.stem + suffix)
        out_path.write_text(json.dumps(data, indent=2))

if __name__ == "__main__":
    splits_dir = Path("yolo_splits")
    mapping = json.loads(Path("/home/serverai/ltdoanh/LayoutGeneration/objectfree/clip_scale_mapping.json").read_text())
    for sub in ("generic", "specific"):
        process_annotations(splits_dir / sub, mapping)
